<a href="https://colab.research.google.com/github/salvapineda/notebooks/blob/main/BranchAndBound.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Branch and bound

This notebook solves the following mixed-integer linear optimization problem:

$$
\begin{align}
\underset{x_1,x_2,x_3,x_4}{\min} \quad & 3x_1+2x_2 \\
\text{s.t.} \quad & x_1-2x_2+x_3 = 5/2\\
& 2x_1+x_2+x_4=3/2\\
& x_1,x_2,x_3,x_4 \geq 0 \\
& x_2, x_3 \in \mathbb{N}
\end{align}
$$

## Requirements

In [1]:
# PYOMO
!pip install pyomo 
import pyomo.environ as pe
# GLPK
!apt-get install -y -qq glpk-utils
glpk = pe.SolverFactory('glpk', executable='/usr/bin/glpsol')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 3.1 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 128275 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.7.1+dfsg-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.7.1+dfsg-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.7.1+dfsg-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.7.1+dfsg-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.7.1+dfsg-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.7.1+dfsg-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-2_amd64.deb ...
Unpacking libgl

## Solving the MIP directly
First we solve the MILP directly using GLPK solver

In [2]:
# Model
m = pe.ConcreteModel()
# Variables
m.x1 = pe.Var(domain=pe.NonNegativeReals)
m.x2 = pe.Var(domain=pe.NonNegativeIntegers)
m.x3 = pe.Var(domain=pe.NonNegativeIntegers)
m.x4 = pe.Var(domain=pe.NonNegativeReals)
# Objective function
m.obj = pe.Objective(expr = 3*m.x1+2*m.x2)
# Constraints 
m.con1 = pe.Constraint(expr = m.x1-2*m.x2+m.x3 == 5/2)
m.con2 = pe.Constraint(expr = 2*m.x1+m.x2+m.x4 == 3/2)
# Solve problem using GLPK solver
glpk.solve(m).write()
# Print
print('obj =',m.obj())
print('x1 =',m.x1.value)
print('x2 =',m.x2.value)
print('x3 =',m.x3.value)
print('x4 =',m.x4.value)


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1.5
  Upper bound: 1.5
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 5
  Number of nonzeros: 7
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 3
      Number of created subproblems: 3
  Error rc: 0
  Time: 0.015077352523803711
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------

## Solving the MIP using branch and bound
Now we solve it using the branch and bound method. First we set the upper- and lower-bound to $+\infty$ and $-\infty$, respectively. Then we solve the corresponding relaxed problem

In [3]:
# Model
m = pe.ConcreteModel()
# Variables
m.x1 = pe.Var(domain=pe.NonNegativeReals)
m.x2 = pe.Var(domain=pe.NonNegativeReals)
m.x3 = pe.Var(domain=pe.NonNegativeReals)
m.x4 = pe.Var(domain=pe.NonNegativeReals)
# Objective function
m.obj = pe.Objective(expr = 3*m.x1+2*m.x2)
# Constraints 
m.con1 = pe.Constraint(expr = m.x1-2*m.x2+m.x3 == 5/2)
m.con2 = pe.Constraint(expr = 2*m.x1+m.x2+m.x4 == 3/2)
# Solve problem using GLPK
glpk.solve(m).write()
# Print
print('obj =',m.obj())
print('x1 =',m.x1.value)
print('x2 =',m.x2.value)
print('x3 =',m.x3.value)
print('x4 =',m.x4.value)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 0.0
  Upper bound: 0.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 5
  Number of nonzeros: 7
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.018999814987182617
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------

First we update the lower-bound to the objective function of the relaxed problem, that is, to 0.0. Variable $x_3$ is not integer. Therefore, we branch and solve two problems. In the first one, we add the constraint $x_3\leq2$

In [4]:
# Model
m = pe.ConcreteModel()
# Variables
m.x1 = pe.Var(domain=pe.NonNegativeReals)
m.x2 = pe.Var(domain=pe.NonNegativeReals)
m.x3 = pe.Var(domain=pe.NonNegativeReals)
m.x4 = pe.Var(domain=pe.NonNegativeReals)
# Objective function
m.obj = pe.Objective(expr = 3*m.x1+2*m.x2)
# Constraints 
m.con1 = pe.Constraint(expr = m.x1-2*m.x2+m.x3 == 5/2)
m.con2 = pe.Constraint(expr = 2*m.x1+m.x2+m.x4 == 3/2)
m.con3 = pe.Constraint(expr = m.x3 <= 2)
# Solve problem using GLPK
glpk.solve(m).write()
# Print
print('obj =',m.obj())
print('x1 =',m.x1.value)
print('x2 =',m.x2.value)
print('x3 =',m.x3.value)
print('x4 =',m.x4.value)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1.5
  Upper bound: 1.5
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 5
  Number of nonzeros: 8
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.019550323486328125
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------

The solution of this problem satisfies integrality. Therefore, we update the upper-bound to the value of the objective function 1.5 and there is no need to branch. We solve next the second problem with the additional constraint $x_3\geq3$

In [5]:
# Model
m = pe.ConcreteModel()
# Variables
m.x1 = pe.Var(domain=pe.NonNegativeReals)
m.x2 = pe.Var(domain=pe.NonNegativeReals)
m.x3 = pe.Var(domain=pe.NonNegativeReals)
m.x4 = pe.Var(domain=pe.NonNegativeReals)
# Objective function
m.obj = pe.Objective(expr = 3*m.x1+2*m.x2)
# Constraints 
m.con1 = pe.Constraint(expr = m.x1-2*m.x2+m.x3 == 5/2)
m.con2 = pe.Constraint(expr = 2*m.x1+m.x2+m.x4 == 3/2)
m.con3 = pe.Constraint(expr = m.x3 >= 3)
# Solve problem using GLPK
glpk.solve(m).write()
# Print
print('obj =',m.obj())
print('x1 =',m.x1.value)
print('x2 =',m.x2.value)
print('x3 =',m.x3.value)
print('x4 =',m.x4.value)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 0.5
  Upper bound: 0.5
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 5
  Number of nonzeros: 8
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.023290634155273438
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------

Varible $x_2$ is not integer. Then we have to branch again and solve two new problems. In the first one, we include the constraint $x_2\leq0$

In [6]:
# Model
m = pe.ConcreteModel()
# Variables
m.x1 = pe.Var(domain=pe.NonNegativeReals)
m.x2 = pe.Var(domain=pe.NonNegativeReals)
m.x3 = pe.Var(domain=pe.NonNegativeReals)
m.x4 = pe.Var(domain=pe.NonNegativeReals)
# Objective function
m.obj = pe.Objective(expr = 3*m.x1+2*m.x2)
# Constraints 
m.con1 = pe.Constraint(expr = m.x1-2*m.x2+m.x3 == 5/2)
m.con2 = pe.Constraint(expr = 2*m.x1+m.x2+m.x4 == 3/2)
m.con3 = pe.Constraint(expr = m.x3 >= 3)
m.con4 = pe.Constraint(expr = m.x2 <= 0)
# Solve problem using GLPK
glpk.solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 5
  Number of nonzeros: 9
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: infeasible
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.0272064208984375


The problem is infeasible and therefore we stop the branching as well. We continue with the second problem with the constraint $x_2\geq1$

In [7]:
# Model
m = pe.ConcreteModel()
# Variables
m.x1 = pe.Var(domain=pe.NonNegativeReals)
m.x2 = pe.Var(domain=pe.NonNegativeReals)
m.x3 = pe.Var(domain=pe.NonNegativeReals)
m.x4 = pe.Var(domain=pe.NonNegativeReals)
# Objective function
m.obj = pe.Objective(expr = 3*m.x1+2*m.x2)
# Constraints 
m.con1 = pe.Constraint(expr = m.x1-2*m.x2+m.x3 == 5/2)
m.con2 = pe.Constraint(expr = 2*m.x1+m.x2+m.x4 == 3/2)
m.con3 = pe.Constraint(expr = m.x3 >= 3)
m.con4 = pe.Constraint(expr = m.x2 >= 1)
# Solve problem using GLPK
glpk.solve(m).write()
# Print
print('obj =',m.obj())
print('x1 =',m.x1.value)
print('x2 =',m.x2.value)
print('x3 =',m.x3.value)
print('x4 =',m.x4.value)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 2.0
  Upper bound: 2.0
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 5
  Number of nonzeros: 9
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.02126288414001465
# ----------------------------------------------------------
#   Solution Information
# -----------------------------------------

Variable $x_3$ is not integer. However, there is no need for branching since the objective value is higher than the current upper-bound, which is 1.5. Therefore we stop the branch and bound and set the solution to 

$$
\begin{align}
&\text{obj} = 1.5 \\
&x_1 = 0.5 \\
&x_2 = 0.0 \\
&x_3 = 2.0 \\
&x_4 = 0.5
\end{align}
$$

